In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

ModuleNotFoundError: No module named 'sql'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

UsageError: Cell magic `%%sql` not found.


## Top Genres Sold in the US

In [ ]:
%%sql
WITH us_tracks AS (
    SELECT il.*
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    WHERE billing_country = "USA"
    )

SELECT
    g.name Genre,
    COUNT(usa.invoice_line_id) "Tracks Sold",
    CAST(COUNT(usa.invoice_line_id) AS float)/
        (SELECT COUNT(*) FROM us_tracks) "Percentage Sold"
FROM us_tracks usa
INNER JOIN track t ON t.track_id = usa.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Recommendation: Top 5 Genre sold in the US - Rock, Alt/Punk, Metal, R&B/Soul and Blues. I would recommend adding Red Tone (Punk) and Slim Jim Bits (Blues) on the procurement list.

## Sales Performance Analysis

In [ ]:
%%sql
WITH sales_table AS (
        SELECT 
            i.customer_id,
            c.support_rep_id,
            SUM(i.total) cust_sales_total
        FROM invoice i
        INNER JOIN customer c ON c.customer_id = i.customer_id
        GROUP BY 1,2
    )

SELECT 
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    SUM(s.cust_sales_total) total_sales 
FROM sales_table s
INNER JOIN employee e ON e.employee_id = s.support_rep_id
GROUP BY 1

While there is a 20% difference in sales between the top ranking sales representative Jane Peacock and the lowest, Steve Johnson, the difference roughly correlates with the differences on when they were hired.

## Analyzing Sales by Country

Categorizes all invoices as either an album purchase or individual tracks in total sales.

In [ ]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Based on the data provided, there may be a market in the following:

- Czech Republic
- United Kingdom
- India

Although the amount of data is not sufficient, the possibility of oppotunity in these countries being a market is relatively high. The marketing campaigns shouldn't be limited to the top 3 customer-countries. New customers from the countries listed above might be promising as the numbers show positive sales. Perhaps small campaigns might help.

## Albums vs Individual Tracks

In [3]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

UsageError: Cell magic `%%sql` not found.


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.